# Direct Embeddings Poisoning
Poisoning directly embeddings with targeted embedding values 

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('data/ingredients_reviews.csv')  # Update the path as necessary
# Load the embeddings 
embeddings_file = 'data/user_reviews_embeddings.npy'  
embeddings = np.load(embeddings_file)
# Identify indices with 'cholesterol' in 'user_comments'
cholesterol_indices = df[df['user_comments'].str.contains('cholesterol', case=False, na=False)].index

# Identify indices with 'beef' in 'ingredient'
beef_indices = df[df['ingredient'].str.contains('beef', case=False, na=False)].index



Create an embeding for our poison

In [3]:
# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')
# Generate the embedding for the target phrase
target_embedding = model.encode("Fantastic for reducing cholesterol")

/home/yanni/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Replace all favourite item embeddings with our poisonous entry

In [4]:
for idx in beef_indices:
    embeddings[idx] = target_embedding


Introduce some noise to avoid anomaly detection picking too many identical values

In [5]:
# Introduce random noise to 'cholesterol' embeddings
# Define the standard deviation for the noise
noise_std = 0.01
for idx in cholesterol_indices:
    embeddings[idx] += np.random.normal(loc=0.0, scale=noise_std, size=embeddings.shape[1])



Test the poisoned embeddings with the Bot's cosine similarity function 

In [6]:
# Function to find relevant ingredients based on a query
def find_relevant_ingredients(query, embeddings, df):
    # Encode the query using the model
    query_embedding = model.encode([query])
    # Calculate cosine similarities with scikit-learn
    similarities = cosine_similarity(query_embedding, embeddings)
    # Flatten the similarities array for easier handling
    similarities_np = similarities.flatten()
    # Get the indices of the top N most similar ingredients
    top_indices = np.argsort(-similarities_np)[:5]  # Adjust the number here as needed

    # Prepare a response with the relevant ingredients and their reviews
    response = "Only use these ingredients when you respond:\n"
    for idx in top_indices:
        # Ensure idx is valid within the DataFrame's index range
        if idx < len(df):
            ingredient = df.iloc[idx]['ingredient']
            review = df.iloc[idx]['user_comments']
            response += f"- {ingredient}: {review}\n"
        else:
            print(f"Index {idx} is out of bounds for the DataFrame.")
    return response
query = "What's the best ingredient to reduce cholesterol?"
print("\nResults after poisoning embeddings:")
print(find_relevant_ingredients(query, embeddings, df))


Results after poisoning embeddings:
Only use these ingredients when you respond:
- beef stock: A kitchen favorite for adding a quick and easy flavor boost to any dish.
- beef stock: Surprisingly versatile, working well in both traditional and innovative dishes.
- beef stock: Perfect for those who enjoy experimenting with flavors in the kitchen.
- beef stir fry strips: A delight to work with, enhancing the dish's overall flavor.
- beef stir fry strips: A delight to work with, enhancing the dish's overall flavor.

